In [1]:
# testing habitat lab setup
import sys
import math
import io
from typing import Dict, Any
import habitat
import numpy as np
import quaternion
from scipy.spatial.transform import Rotation
from habitat.core.utils import try_cv2_import
from habitat.utils.visualizations import maps
from torchlightning_utils import cli
import matplotlib.pyplot as plt
from PIL import Image

cv2 = try_cv2_import()

# todo: implement metrics
# todo: pose and maps transform


def draw_top_down_map(info, output_size):
    return maps.colorize_draw_agent_and_fit_to_height(info["top_down_map"], output_size)


def map_centered_by_agent(info) -> np.ndarray:
    """Center the map wrt to agent's pose.

    Args:
        info (dict): _description_
    """
    env_map = info["top_down_map"]["map"]
    agent_map_pose = np.array(
        [
            info["top_down_map"]["agent_map_coord"][0],
            info["top_down_map"]["agent_map_coord"][1],
            info["top_down_map"]["agent_angle"],
        ]
    )
    cx = agent_map_pose[0]
    cy = agent_map_pose[1]
    map_size = min(info["top_down_map"]["map"].shape)
    rect = (
        (cx, cy),
        (map_size, map_size),
        90 - math.degrees(agent_map_pose[2]),
    )
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    # get width and height of the detected rectangle
    width = int(rect[1][0])
    height = int(rect[1][1])
    src_pts = box.astype("float32")
    # coordinate of the points in box points after the rectangle has been
    # straightened
    dst_pts = np.array(
        [[0, height - 1], [0, 0], [width - 1, 0], [width - 1, height - 1]],
        dtype="float32",
    )
    # the perspective transformation matrix
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    # directly warp the rotated rectangle to get the straightened rectangle
    warped_map = cv2.warpPerspective(env_map, M, (width, height))
    return warped_map


def get_2dpose(env: habitat.Env):
    """Return agent's pose in both simulation env and also wrt map.

    Args:
        env (habitat.Env): current environment

    Returns:
        dict:
            pose (np.ndarray): agent's pose wrt simulation env, 4d with 1-3d representing position, last entry represents yaw
    """
    agent_state = env._sim.get_agent_state()
    rotation_angles = Rotation.from_quat(
        quaternion.as_float_array(agent_state.rotation)
    ).as_euler("ZYX", degrees=False)
    pose = np.concatenate(
        [
            agent_state.position,
            [rotation_angles[1]],
        ]
    )

    return pose


def pltimg(img: np.ndarray) -> np.ndarray:
    """Given an image (not rgb), use plt to transform it image rgb image

    Args:
        img (np.ndarray): image with leading two dims should be width and height.

    Returns:
        np.ndarray: rgb repr of img with size (original w, original h, 3)
    """
    fig = plt.figure()
    fig.add_axes([0, 0, 1, 1])  # remove margins
    plt.imshow(img)
    buf = io.BytesIO()
    plt.axis("off")
    plt.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    pil_img = Image.open(buf).convert("RGB").resize((img.shape[0], img.shape[1]))
    buf.close()
    np_img = np.asarray(pil_img)
    return np_img


def make_image(observations, info):
    rgb_im = observations["rgb"]
    depth_im = pltimg(np.squeeze(observations["depth"], axis=2))
    semantic_im = pltimg(np.squeeze(observations["semantic"], axis=2))
    top_down_map = draw_top_down_map(info, rgb_im.shape[0])
    output_im = np.concatenate((rgb_im, depth_im, semantic_im, top_down_map), axis=1)
    return output_im


/datadrive/anaconda3/envs/habitat/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# run episode
config_path = "/home/azureuser/AutonomousSystemsResearch/habitat-simulation/configs/collect.yaml"
with open(config_path, mode="r") as f:
    from omegaconf import OmegaConf
    config = OmegaConf.create(f.read())
    config = OmegaConf.to_container(config, resolve=True)

offline_dataset_config, online_env_config = config["offline_dataset"], config["online_env"]


env_config = habitat.get_config(config_paths=online_env_config["env_config_path"])    

with habitat.config.read_write(env_config):
    env_config.habitat.dataset.split = online_env_config["env_split"]
    env_config.habitat.dataset.data_path = online_env_config["env_data_path"]
    env_config.habitat.dataset.scenes_dir = online_env_config["env_scene_dir"]
    env_config.habitat.simulator.scene_dataset = online_env_config[
        "env_scene_dataset_config"
    ]

In [3]:
# create env
env = habitat.Env(config=env_config)

2023-02-03 00:07:45,643 Initializing dataset PointNav-v1
2023-02-03 00:07:45,824 initializing sim Sim-v0


: 

: 

In [17]:
from habitat.sims import make_sim
env.current_episode.scene_dataset_config = online_env_config[
        "env_scene_dataset_config"
    ]
env._config.defrost()
env._config.simulator.scene_dataset = (
                env.current_episode.scene_dataset_config
            )
env._config.freeze()
print(env._config)
env_sim = make_sim(
           id_sim=env._config.simulator.type, config=env._config.simulator
       )

2022-11-24 01:28:12,040 initializing sim Sim-v0


dataset:
  content_scenes: ['*']
  data_path: /datadrive/azure_storage/pactdata/habitat-data/habitat-dataset/hm3d/pointnav/train/train.json.gz
  scenes_dir: /datadrive/azure_storage/pactdata/habitat-data/habitat-dataset
  split: train
  type: PointNav-v1
env_task: GymHabitatEnv
env_task_gym_dependencies: []
env_task_gym_id: 
environment:
  iterator_options:
    cycle: True
    group_by_scene: True
    max_scene_repeat_episodes: -1
    max_scene_repeat_steps: 10000
    num_episode_sample: -1
    shuffle: True
    step_repetition_range: 0.2
  max_episode_seconds: 10000000
  max_episode_steps: 500
gym:
  achieved_goal_keys: []
  action_keys: None
  auto_name: 
  desired_goal_keys: []
  obs_keys: None
pyrobot:
  base_controller: proportional
  base_planner: none
  bump_sensor:
    type: PyRobotBumpSensor
  depth_sensor:
    center_crop: False
    height: 480
    max_depth: 5.0
    min_depth: 0.0
    normalize_depth: True
    type: PyRobotDepthSensor
    width: 640
  locobot:
    actions: [

In [18]:
def print_scene_recur(scene, limit_output=10):
    print(f"House has {len(scene.levels)} levels, {len(scene.regions)} regions and {len(scene.objects)} objects")
    print(f"House center:{scene.aabb.center} dims:{scene.aabb.sizes}")

    count = 0
    for level in scene.levels:
        print(
            f"Level id:{level.id}, center:{level.aabb.center},"
            f" dims:{level.aabb.sizes}"
        )
        for region in level.regions:
            print(
                f"Region id:{region.id}, category:{region.category.name()},"
                f" center:{region.aabb.center}, dims:{region.aabb.sizes}"
            )
            for obj in region.objects:
                print(
                    f"Object id:{obj.id}, category:{obj.category.name()},"
                    f" center:{obj.aabb.center}, dims:{obj.aabb.sizes}"
                )
                count += 1
                if count >= limit_output:
                    return None

# Print semantic annotation information (id, category, bounding box details)
# about levels, regions and objects in a hierarchical fashion
scene = env._sim.semantic_scene
print_scene_recur(scene)

House has 0 levels, 0 regions and 0 objects
House center:[0. 0. 0.] dims:[-inf -inf -inf]


In [7]:
# simple agent from native habitat
goal_radius = env.episodes[0].goals[0].radius
if goal_radius is None:
    goal_radius = env_config.habitat.simulator.forward_step_size
# agent = PACTPointNavAgent(
#     raw_action_space=env.habitat_env.action_space, agent_config=agent_config
# )
agent = ShortestPathFollowerAgent(env._sim, goal_radius)
print("Environment creation successful")

for episode in range(online_env_config["num_episodes"]):
    env.current_episode.scene_dataset_config = online_env_config[
        "env_scene_dataset_config"
    ]

    observations = env.reset()
    env.current_episode.scene_dataset_config = online_env_config[
        "env_scene_dataset_config"
    ]
    with habitat.config.read_write(env_config):
        env_config.habitat.simulator.scene_dataset = online_env_config[
            "env_scene_dataset_config"
        ]
    dataset_collector.episode_start(
        env.current_episode.__getstate__()
    )

    # RLEnv.reset() only returns observations: https://aihabitat.org/docs/habitat-lab/habitat.core.env.RLEnv.html
    step = 0
    while not env.episode_over:
        # action = agent.act(observations)
        action = agent.act(env.current_episode.goals[0].position)
        if action is None:
            break

        next_observations = env.step(action)
        print(next_observations.keys())
        observations = next_observations

        step += 1

env.close()

AttributeError: simulator

House has 0 levels, 0 regions and 0 objects
House center:[0. 0. 0.] dims:[-inf -inf -inf]


In [10]:
print(agent_cfg)

AgentConfiguration(height=1.5, radius=0.1, sensor_specifications=[<habitat_sim._ext.habitat_sim_bindings.CameraSensorSpec object at 0x7f2a63a95680>], action_space={'move_forward': ActionSpec(name='move_forward', actuation=ActuationSpec(amount=0.25, constraint=None)), 'turn_left': ActionSpec(name='turn_left', actuation=ActuationSpec(amount=10.0, constraint=None)), 'turn_right': ActionSpec(name='turn_right', actuation=ActuationSpec(amount=10.0, constraint=None))}, body_type='cylinder')


action move_forward
{'semantic': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32), 'collided': False}
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
action move_forward
{'semantic': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32), 'collided': False}
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
action move_forward
{'semantic': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 